In [2]:
import pandas as pd
import tensorflow as tf  # Importa o TensorFlow para construção de modelos de aprendizado de máquina
import sklearn
import scikeras

In [4]:
from scikeras.wrappers import KerasClassifier  # Importa KerasClassifier do scikeras para usar com o modelo Keras
from tensorflow.keras import backend as k  # Importa o backend Keras para gerenciamento de sessões
from tensorflow.keras.models import Sequential  # Importa a classe Sequential para criar uma rede neural sequencial
from sklearn.model_selection import GridSearchCV  # Importa a classe GridSearchCV para otimização de hiperparâmetros


In [12]:
X = pd.read_csv('entradas_breast.csv')
y = pd.read_csv('saidas_breast.csv')

In [13]:

def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
    """
    Cria uma rede neural sequencial para classificação binária com parâmetros configuráveis.

    Args:
        optimizer: Otimizador a ser utilizado durante o treinamento do modelo.
        loss: Função de perda a ser utilizada na compilação do modelo.
        kernel_initializer: Inicializador dos pesos das camadas da rede.
        activation: Função de ativação a ser utilizada nas camadas ocultas.
        units (int): Número de unidades nas camadas ocultas.

    Returns:
        Sequential: A rede neural configurada.
    """
    # Limpa qualquer sessão anterior do Keras para evitar conflitos com modelos anteriores
    k.clear_session()
    # Cria uma nova rede neural sequencial
    rede_neural = Sequential([
        # Camada de entrada que recebe dados com 30 características
        tf.keras.layers.InputLayer(shape=(30,)),  # Define a forma da entrada com 30 características
        
        # Primeira camada oculta: camada densa com 'units' neurônios e função de ativação especificada
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        
        # Camada Dropout para regularização, ajudando a prevenir overfitting
        tf.keras.layers.Dropout(rate=0.2),  # Define a taxa de dropout
        
        # Segunda camada oculta: outra camada densa com 'units' neurônios e função de ativação especificada
         tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        
        # Outra camada Dropout para regularização
        tf.keras.layers.Dropout(rate=0.2),  # Define a taxa de dropout
        
        # Camada de saída: camada densa com 1 neurônio para previsão binária (0 ou 1)
        tf.keras.layers.Dense(units=1, activation='sigmoid')  # Usando sigmoide para obter uma probabilidade
    ])
    
    # Compila o modelo da rede neural, definindo a função de perda, o otimizador e as métricas de desempenho
    rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])  # Configura a perda e o otimizador
    
    return rede_neural  # Retorna a rede neural criada


In [14]:
# Cria uma instância de KerasClassifier
rede_neural = KerasClassifier(model = criar_rede)

In [15]:

# Define um dicionário com os parâmetros a serem testados na busca em grade
parametros = {
    'batch_size': [10, 30],  # Tamanhos de lote a serem testados
    'epochs': [50, 100],  # Números de épocas para treinamento
    'model__optimizer': ['adam', 'sgd'],  # Diferentes otimizadores a serem testados
    'model__loss': ['binary_crossentropy', 'hinge'],  # Funções de perda para testar
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],  # Funções de ativação para testar
    'model__neurons': [16, 8],  # Número de unidades nas camadas ocultas
}



In [16]:
parametros

{'batch_size': [10, 30],
 'epochs': [50, 100],
 'model__optimizer': ['adam', 'sgd'],
 'model__loss': ['binary_crossentropy', 'hinge'],
 'model__kernel_initializer': ['random_uniform', 'normal'],
 'model__activation': ['relu', 'tanh'],
 'model__neurons': [16, 8]}

In [17]:
# Cria uma instância de GridSearchCV para encontrar a melhor combinação de hiperparâmetros
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                          scoring = 'accuracy', cv = 5)

In [18]:
# Fit do GridSearchCV nos dados de treinamento
grid_search = grid_search.fit(X, y)  # X e y devem ser definidos anteriormente como os dados de treinamento

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 832us/step - binary_accuracy: 0.5964 - loss: 0.8555
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - binary_accuracy: 0.6600 - loss: 0.5980
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - binary_accuracy: 0.7531 - loss: 0.5242
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - binary_accuracy: 0.7620 - loss: 0.5227
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - binary_accuracy: 0.7748 - loss: 0.4605
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - binary_accuracy: 0.7433 - loss: 0.4406
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - binary_accuracy: 0.8369 - loss: 0.3770
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - binary_accuracy: 0.8271 - loss: 0.3683
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - binary_accuracy: 0.8584 - loss: 0.3608
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - binary_accuracy: 0.8322 - loss: 0.3471
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - binary_accurac

KeyboardInterrupt: 

In [135]:
# Obtém os melhores parâmetros encontrados pelo GridSearchCV
melhores_parametros = grid_search.best_params_

# Obtém o melhor score (pontuação) médio encontrado pelo GridSearchCV durante a validação cruzada
melhor_score = grid_search.best_score_

# Imprime os melhores parâmetros encontrados
print(melhores_parametros)

# Imprime a melhor pontuação média encontrada
print(melhor_score)

# Imprime o objeto grid_search para detalhes adicionais, incluindo todos os resultados da busca
print(grid_search)


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'